In [1]:
import numpy as np
from fast_flights import FlightData, Passengers, create_filter, get_flights, helper, Bags
import pickle
import datetime
import os.path
import time
import pandas as pd

In [2]:
domestic_city_pairs = [
    ["ATL","SEA"], 
    ["ATL","DEN"], 
    ["ATL","LIH"],
    ["ATL","OGG"],
    ["ATL","SAN"], 
    ["ATL","SFO"], 
    ["ATL","DFW"],
    ["ATL","LAS"],
    ["ATL","PHX"], 
    ["ATL","BOS"], 
]
num_days = 365
intl_city_pairs = [
    ["ATL","FCO",[7,9,11,13]], 
    ["ATL","DEL",[7,9,11,13]], 
    ["ATL","CDG",[7,9,11,13]],
    ["ATL","FRA",[7,9,11,13]],
    ["ATL","LHR",[7,9,11,13]], 
    ["ATL","ATH",[7,9,11,13]], 
    ["ATL","CAI",[7,9,11,13]],
    ["ATL","CUN",[7,9,11,13]],
    ["ATL","HND",[7,9,11,13]],
]
folder_path = 'logs/'
date_today_file = datetime.datetime.today().strftime('%Y-%m-%d') + '.pkl'
if os.path.isfile(folder_path+date_today_file):
    with open(folder_path+date_today_file, 'rb') as handle:
        itin_dict = pickle.load(handle) 
else:
    itin = helper.gen_itineraries(domestic_city_pairs, "domestic", num_days)
    itin_dict = helper.gen_dict_from_itin(itin)
    with open(folder_path+date_today_file, 'wb') as handle:
        pickle.dump(itin_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_prog_count=0
for key in itin_dict:
    if itin_dict[key]!=0:
        continue
    if save_prog_count==10:
        save_prog_count = 0
        helper.save_prog(itin_dict, folder_path+date_today_file)
    start = time.time()
    origin, destination, departure_date = key.split('_')
    departure_day, departure_month, departure_year = departure_date.split('-')
    days_ahead = (datetime.datetime.strptime(departure_date, "%Y-%m-%d") - datetime.datetime.today()).days
    new_dict={}
    new_dict=helper.initialize_dict(new_dict)
    # Create a new filter
    cookies = { "CONSENT": "YES+" }
    filter = create_filter(
        flight_data=[
            # Include more if it's not a one-way trip
            FlightData(
                date=departure_date,  # Date of departure
                from_airport=origin, 
                to_airport=destination
            ),
            # ... include more for round trips and multi-city trips
        ],
        bags=Bags(
            carryon=1,
            checked=0
        ),
        trip="round-trip",  # Trip (round-trip, one-way, multi-city)
        seat="economy",  # Seat (economy, premium-economy, business or first)
        passengers=Passengers(
            adults=1,
            children=0,
            infants_in_seat=0,
            infants_on_lap=0
        ),
    )
    result = get_flights(filter, cookies=cookies)
    if result == []:
        print("Will retry, exception occured")
        continue
    for i in range(len(result.flights)):
        fl = result.flights[i]
        new_dict = helper.append_itin_to_dict(new_dict, fl, departure_day, departure_month, departure_year,\
                                             origin, destination, days_ahead)
    itin_dict[key] = new_dict
    save_prog_count+=1
    end = time.time()
    print(key," completed")
    print("time_elapsed: ",end-start)


flight_data=[FlightData(date='2024-05-31', from_airport=ATL, to_airport=SEA)],bags=Bags(carryon=1, checked=0),trip=1,passengers=Passengers((1, 0, 0, 0)),seat=1)
https://www.google.com/travel/flights?tfs=GhoSCjIwMjQtMDUtMzFqBRIDQVRMcgUSA1NFQUIBAUgBagIQAZgBAQ%3D%3D&hl=en&tfu=EgQIABABIgA
<Response [200]>
ATL_SEA_2024-05-31  completed
time_elapsed:  1.5639617443084717
